In [1]:
import os
import random
import numpy as np
import pandas as pd
import os, json, joblib
import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import polars as pl
from pathlib import Path
import warnings 
warnings.filterwarnings("ignore")


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.utils.class_weight import compute_class_weight

from sklearn.model_selection import StratifiedGroupKFold
from scipy.spatial.transform import Rotation as R

In [2]:
# (Competition metric will only be imported when TRAINing)
TRAIN = True                     # ← set to True when you want to train
RAW_DIR = Path("C:\\Users\\konno\\SynologyDrive\\datasciense\\projects_foler\\1_kaggle\\CMI\\cmi-detect-behavior-with-sensor-data")

# used when TRAIN=False
PRETRAINED_DIR = Path("C:\\Users\\konno\\SynologyDrive\\datasciense\\projects_foler\\1_kaggle\\CMI\\cmi-detect-behavior-with-sensor-data\\pretrained-model")
EXPORT_DIR = PRETRAINED_DIR                                  # artefacts will be saved here
BATCH_SIZE = 256 #64
PAD_PERCENTILE = 95
LR_INIT = 5e-4
WD = 3e-3
MIXUP_ALPHA = 0.4
EPOCHS = 100 #160
PATIENCE = 40
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


print("▶ imports ready · torch", torch.__version__, "device :", device)

▶ imports ready · torch 2.7.1+cu128 device : cuda


In [3]:
def seed_everything(seed=42):
    os.environ["PYTHONHASHSEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if using multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [4]:
def time_sum(x):
    return x.sum(dim=1)

def squeeze_last_axis(x):
    return x.squeeze(-1)

def expand_last_axis(x):
    return x.unsqueeze(-1)

class SEBlock1D(nn.Module):
    def __init__(self, channels, reduction=8):
        super().__init__()
        self.global_avg_pool = nn.AdaptiveAvgPool1d(1)
        self.fc1 = nn.Linear(channels, channels // reduction)
        self.fc2 = nn.Linear(channels // reduction, channels)

    def forward(self, x):
        # x shape: (batch, channels, time)
        b, c, _ = x.size()
        y = self.global_avg_pool(x).view(b, c)
        y = F.relu(self.fc1(y))
        y = torch.sigmoid(self.fc2(y)).view(b, c, 1)
        return x * y.expand_as(x)

class ResidualSEBlock1D(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, pool_size=2, dropout=0.3):
        super().__init__()
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size, padding=kernel_size // 2, bias=False)
        self.bn1 = nn.BatchNorm1d(out_channels)
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size, padding=kernel_size // 2, bias=False)
        self.bn2 = nn.BatchNorm1d(out_channels)
        self.se = SEBlock1D(out_channels)
        
        self.match_channels = None
        if in_channels != out_channels:
            self.match_channels = nn.Sequential(
                nn.Conv1d(in_channels, out_channels, kernel_size=1, padding=0, bias=False),
                nn.BatchNorm1d(out_channels)
            )
        
        self.pool = nn.MaxPool1d(pool_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        identity = x
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.se(out)

        if self.match_channels is not None:
            identity = self.match_channels(identity)
        
        out = F.relu(out + identity)
        out = self.pool(out)
        out = self.dropout(out)
        return out

class AttentionLayer(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.attn_fc = nn.Linear(input_dim, 1)

    def forward(self, x):
        # x shape: (batch, time, features)
        # Compute scores with tanh activation
        scores = torch.tanh(self.attn_fc(x))  # (batch, time, 1)
        scores = scores.squeeze(-1)           # (batch, time)

        # Softmax over time dimension to get weights
        weights = F.softmax(scores, dim=1)    # (batch, time)
        weights = weights.unsqueeze(-1)       # (batch, time, 1)

        # Weighted sum of input features over time
        context = (x * weights).sum(dim=1)    # (batch, features)
        return context

In [5]:
def preprocess_sequence(df_seq: pd.DataFrame, feature_cols: list[str], scaler: StandardScaler):
    mat = df_seq[feature_cols].ffill().bfill().fillna(0).values
    return scaler.transform(mat).astype('float32')

# MixUp the data argumentation in order to regularize the neural network. 

class MixupGenerator(Dataset):
    def __init__(self, X, y, alpha=0.2):
        """  
        X: np.array or totch.Tensor of shape (N, ...)
        y: np.array or torch.Tensor of shape (N, ...)
        alpha: Beta distribution parameter for mixup
        """
        self.X = torch.tensor(X, dtype=torch.float32) if isinstance(X, np.ndarray) else X
        self.y = torch.tensor(y, dtype=torch.float32) if isinstance(y, np.ndarray) else y
        self.alpha = alpha
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, index):
        # Get current sample
        x1, y1 = self.X[index], self.y[index]

        # Choose a random other sample
        mix_index = np.random.randint(0, len(self.X))
        x2, y2 = self.X[mix_index], self.y[mix_index]

        # Mix them
        lam = np.random.beta(self.alpha, self.alpha)
        x_mix = lam * x1 + (1 - lam) * x2
        y_mix = lam * y1 + (1 - lam) * y2

        return x_mix, y_mix
    
# dataset = MixupGenerator(X_train, y_train, alpha=0.2)
# loader = DataLoader(dataset, batch_size=32, shuffle=True)

In [6]:
def remove_gravity_from_acc(acc_data, rot_data):

    if isinstance(acc_data, pd.DataFrame):
        acc_values = acc_data[['acc_x', 'acc_y', 'acc_z']].values
    else:
        acc_values = acc_data

    if isinstance(rot_data, pd.DataFrame):
        quat_values = rot_data[['rot_x', 'rot_y', 'rot_z', 'rot_w']].values
    else:
        quat_values = rot_data

    num_samples = acc_values.shape[0]
    linear_accel = np.zeros_like(acc_values)
    
    gravity_world = np.array([0, 0, 9.81])

    for i in range(num_samples):
        if np.all(np.isnan(quat_values[i])) or np.all(np.isclose(quat_values[i], 0)):
            linear_accel[i, :] = acc_values[i, :] 
            continue

        try:
            rotation = R.from_quat(quat_values[i])
            gravity_sensor_frame = rotation.apply(gravity_world, inverse=True)
            linear_accel[i, :] = acc_values[i, :] - gravity_sensor_frame
        except ValueError:
             linear_accel[i, :] = acc_values[i, :]
             
    return linear_accel

def calculate_angular_velocity_from_quat(rot_data, time_delta=1/200): # Assuming 200Hz sampling rate
    if isinstance(rot_data, pd.DataFrame):
        quat_values = rot_data[['rot_x', 'rot_y', 'rot_z', 'rot_w']].values
    else:
        quat_values = rot_data

    num_samples = quat_values.shape[0]
    angular_vel = np.zeros((num_samples, 3))

    for i in range(num_samples - 1):
        q_t = quat_values[i]
        q_t_plus_dt = quat_values[i+1]

        if np.all(np.isnan(q_t)) or np.all(np.isclose(q_t, 0)) or \
           np.all(np.isnan(q_t_plus_dt)) or np.all(np.isclose(q_t_plus_dt, 0)):
            continue

        try:
            rot_t = R.from_quat(q_t)
            rot_t_plus_dt = R.from_quat(q_t_plus_dt)

            # Calculate the relative rotation
            delta_rot = rot_t.inv() * rot_t_plus_dt
            
            # Convert delta rotation to angular velocity vector
            # The rotation vector (Euler axis * angle) scaled by 1/dt
            # is a good approximation for small delta_rot
            angular_vel[i, :] = delta_rot.as_rotvec() / time_delta
        except ValueError:
            # If quaternion is invalid, angular velocity remains zero
            pass
            
    return angular_vel

def calculate_angular_distance(rot_data):
    if isinstance(rot_data, pd.DataFrame):
        quat_values = rot_data[['rot_x', 'rot_y', 'rot_z', 'rot_w']].values
    else:
        quat_values = rot_data

    num_samples = quat_values.shape[0]
    angular_dist = np.zeros(num_samples)

    for i in range(num_samples - 1):
        q1 = quat_values[i]
        q2 = quat_values[i+1]

        if np.all(np.isnan(q1)) or np.all(np.isclose(q1, 0)) or \
           np.all(np.isnan(q2)) or np.all(np.isclose(q2, 0)):
            angular_dist[i] = 0 # Или np.nan, в зависимости от желаемого поведения
            continue
        try:
            # Преобразование кватернионов в объекты Rotation
            r1 = R.from_quat(q1)
            r2 = R.from_quat(q2)

            # Вычисление углового расстояния: 2 * arccos(|real(p * q*)|)
            # где p* - сопряженный кватернион q
            # В scipy.spatial.transform.Rotation, r1.inv() * r2 дает относительное вращение.
            # Угол этого относительного вращения - это и есть угловое расстояние.
            relative_rotation = r1.inv() * r2
            
            # Угол rotation vector соответствует угловому расстоянию
            # Норма rotation vector - это угол в радианах
            angle = np.linalg.norm(relative_rotation.as_rotvec())
            angular_dist[i] = angle
        except ValueError:
            angular_dist[i] = 0 # В случае недействительных кватернионов
            pass
            
    return angular_dist

In [7]:
class TwoBranchGestureModel(nn.Module):
    def __init__(self, imu_dim, tof_dim, n_classes, wd=1e-4):
        super().__init__()
        self.imu_dim = imu_dim
        self.tof_dim = tof_dim
        
        # IMU deep branch
        self.imu_branch = nn.Sequential(
            ResidualSEBlock1D(imu_dim, 64, kernel_size=3, dropout=0.1),
            ResidualSEBlock1D(64, 128, kernel_size=5, dropout=0.1)
        )

        # TOF Lighter branch
        self.tof_branch = nn.Sequential(
            nn.Conv1d(tof_dim, 64, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Dropout(0.2),
            nn.Conv1d(64, 128, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm1d(128),
            nn.MaxPool1d(2),
            nn.Dropout(0.2),
        )

        self.lstm = nn.LSTM(256, 128, batch_first=True, bidirectional=True)
        self.gru = nn.GRU(256, 128, batch_first=True, bidirectional=True)

        # Gaussian noise (manual) and projection
        self.projection = nn.Sequential(
            nn.Dropout(0.09),
            nn.Linear(256, 16),
            nn.ELU()
        )

        self.pre_attn_dropout = nn.Dropout(0.4)
        self.attn = AttentionLayer(128*2 + 128*2 + 16)

        # Dense layer
        self.mlp = nn.Sequential(
            nn.Linear(528, 256, bias=False),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0,5),
            nn.Linear(256, 128, bias=False),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, n_classes),  # Softmax handled by loss (e.g., CrossEntropyLoss)
        )
    
    def forward(self, x):
        imu = x[:, :, :self.imu_dim].permute(0, 2, 1) #(B, imu_dim, T)
        tof = x[: ,:, self.imu_dim:].permute(0, 2, 1) #(B, tof_dim, T)

        imu_feat = self.imu_branch(imu)
        tof_feat = self.tof_branch(tof)

        imu_feat = imu_feat.permute(0, 2, 1)
        tof_feat = tof_feat.permute(0, 2, 1)

        merged = torch.cat([imu_feat, tof_feat], dim=-1)  #(B, T', 256)

        xa, _ = self.lstm(merged)
        xb, _ = self.gru(merged)
        xc = self.projection(merged)

        x_cat = torch.cat([xa, xb, xc], dim=-1)  #(B, T', 512)
        x_cat = self.pre_attn_dropout(x_cat)
        context = self.attn(x_cat)

        return self.mlp(context)

In [8]:
### DATA CREATION
print("▶ TRAIN MODE – loading dataset …")
df = pd.read_csv(RAW_DIR / "train.csv")

train_dem_df = pd.read_csv(RAW_DIR / "train_demographics.csv")
df_for_groups = pd.merge(df.copy(), train_dem_df, on='subject', how='left')

le = LabelEncoder()
df['gesture_int'] = le.fit_transform(df['gesture'])
np.save(EXPORT_DIR / "gesture_classes.npy", le.classes_)
gesture_classes = le.classes_

print(" Calculating base engineered IMU features (magnitude, angle) ...")
df['acc_mag'] = np.sqrt(df['acc_x']**2 + df['acc_y']**2 + df['acc_z']**2)
df['rot_angle'] = 2* np.arccos(df['rot_w'].clip(-1, 1))

print(" Calculating engineered IMU derivatives (jerk, angular velocity) for original acc_mag ...")
df['acc_mag_jerk'] = df.groupby('sequence_id')['acc_mag'].diff().fillna(0)
df['rot_angle_vel'] = df.groupby('sequence_id')['rot_angle'].diff().fillna(0)

print(" Removing gravity and calculating linear acceleration features...")

linear_accel_list = []
for _, group in df.groupby('sequence_id'):
    acc_data_group = group[['acc_x', 'acc_y', 'acc_z']]
    rot_data_group = group[['rot_x', 'rot_y', 'rot_z', 'rot_w']]
    linear_accel_group = remove_gravity_from_acc(acc_data_group, rot_data_group)
    linear_accel_list.append(pd.DataFrame(linear_accel_group, columns=['linear_acc_x', 'linear_acc_y', 'linear_acc_z'], index=group.index))

df_linear_accel = pd.concat(linear_accel_list)
df = pd.concat([df, df_linear_accel], axis=1)

df['linear_acc_mag'] = np.sqrt(df['linear_acc_x']**2 + df['linear_acc_y']**2 + df['linear_acc_z']**2)
df['linear_acc_mag_jerk'] = df.groupby('sequence_id')['linear_acc_mag'].diff().fillna(0)

print("  Calculating angular velocity from quaternion derivatives...")
angular_vel_list = []
for _, group in df.groupby('sequence_id'):
    rot_data_group = group[['rot_x', 'rot_y', 'rot_z', 'rot_w']]
    angular_vel_group = calculate_angular_velocity_from_quat(rot_data_group)
    angular_vel_list.append(pd.DataFrame(angular_vel_group, columns=['angular_vel_x', 'angular_vel_y', 'angular_vel_z'], index=group.index))

df_angular_vel = pd.concat(angular_vel_list)
df = pd.concat([df, df_angular_vel], axis=1)

print("  Calculating angular distance between successive quaternions...")
angular_distance_list = []
for _, group in df.groupby('sequence_id'):
    rot_data_group = group[['rot_x', 'rot_y', 'rot_z', 'rot_w']]
    angular_dist_group = calculate_angular_distance(rot_data_group)
    angular_distance_list.append(pd.DataFrame(angular_dist_group, columns=['angular_distance'], index=group.index))

df_angular_distance = pd.concat(angular_distance_list)
df = pd.concat([df, df_angular_distance], axis=1)

meta_cols = { } # This was an empty dict in your provided code, keeping it as is.

imu_cols_base = ['linear_acc_x', 'linear_acc_y', 'linear_acc_z']
imu_cols_base.extend([c for c in df.columns if c.startswith('rot_') and c not in ['rot_angle', 'rot_angle_vel']])

imu_engineered_features = [
    'acc_mag', 'rot_angle',
    'acc_mag_jerk', 'rot_angle_vel',
    'linear_acc_mag', 'linear_acc_mag_jerk',
    'angular_vel_x', 'angular_vel_y', 'angular_vel_z', # Existing new features
    'angular_distance' # Added new feature
]
imu_cols = imu_cols_base + imu_engineered_features
imu_cols = list(dict.fromkeys(imu_cols)) # Для удаления дубликатов

thm_cols_original = [c for c in df.columns if c.startswith('thm_')]

## tof data
tof_aggregated_cols_template = []
for i in range(1, 6):
    tof_aggregated_cols_template.extend([f'tof_{i}_mean', f'tof_{i}_std', f'tof_{i}_min', f'tof_{i}_max'])

final_feature_cols = imu_cols + thm_cols_original + tof_aggregated_cols_template
imu_dim_final = len(imu_cols)
tof_thm_aggregated_dim_final = len(thm_cols_original) + len(tof_aggregated_cols_template)

print(f"  IMU (incl. engineered & derivatives) {imu_dim_final} | THM + Aggregated TOF {tof_thm_aggregated_dim_final} | total {len(final_feature_cols)} features")
np.save(EXPORT_DIR / "feature_cols.npy", np.array(final_feature_cols))

print("  Building sequences with aggregated TOF and preparing data for scaler...")
seq_gp = df.groupby('sequence_id') 

all_steps_for_scaler_list = []
X_list_unscaled, y_list_int_for_stratify, lens = [], [], [] 

for seq_id, seq_df_orig in seq_gp:
    seq_df = seq_df_orig.copy()

    for i in range(1, 6):
        pixel_cols_tof = [f"tof_{i}_v{p}" for p in range(64)]
        tof_sensor_data = seq_df[pixel_cols_tof].replace(-1, np.nan)
        seq_df[f'tof_{i}_mean'] = tof_sensor_data.mean(axis=1)
        seq_df[f'tof_{i}_std']  = tof_sensor_data.std(axis=1)
        seq_df[f'tof_{i}_min']  = tof_sensor_data.min(axis=1)
        seq_df[f'tof_{i}_max']  = tof_sensor_data.max(axis=1)
    
    mat_unscaled = seq_df[final_feature_cols].ffill().bfill().fillna(0).values.astype('float32')
    
    all_steps_for_scaler_list.append(mat_unscaled)
    X_list_unscaled.append(mat_unscaled)
    y_list_int_for_stratify.append(seq_df['gesture_int'].iloc[0])
    lens.append(len(mat_unscaled))

# fit scaler
print("  Fitting StandardScaler...")
all_steps_concatenated = np.concatenate(all_steps_for_scaler_list, axis=0)
scaler = StandardScaler().fit(all_steps_concatenated)
joblib.dump(scaler, EXPORT_DIR / "scaler.pkl")
del all_steps_for_scaler_list, all_steps_concatenated

# scale individual sequences
print("  Scaling and padding sequences...")
X_scaled_list = [scaler.transform(x_seq) for x_seq in X_list_unscaled]
del X_list_unscaled

# calculate pad length
pad_len = int(np.percentile(lens, PAD_PERCENTILE))
np.save(EXPORT_DIR / "sequence_maxlen.npy", pad_len)

# post pad or truncate sequences
def pad_or_truncate(seq, max_len, dtype=np.float32):
    seq = seq[:max_len] # truncate if too long
    padded = np.zeros((max_len, seq.shape[1]), dtype=dtype)
    padded[:len(seq)] = seq
    return padded

# padding section
X_padded_np = np.stack([pad_or_truncate(seq, pad_len) for seq in X_scaled_list])
X = torch.tensor(X_padded_np, dtype=torch.float32)  # shape: (N, T, D)
del X_scaled_list

y_int_for_stratify = np.array(y_list_int_for_stratify)
y = F.one_hot(torch.tensor(y_int_for_stratify), num_classes=len(le.classes_)).float()

print("  Splitting data and preparing for training...")
X_tr, X_val, y_tr, y_val = train_test_split(X, y, test_size=0.2, random_state=82, stratify=y_int_for_stratify)

cw_vals = compute_class_weight('balanced', classes=np.arange(len(le.classes_)), y=y_int_for_stratify)

# Data Loader
train_dataset = MixupGenerator(X_tr, y_tr, alpha=0.2)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

val_dataset = MixupGenerator(X_val, y_val, alpha=0.2)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True)


▶ TRAIN MODE – loading dataset …
 Calculating base engineered IMU features (magnitude, angle) ...
 Calculating engineered IMU derivatives (jerk, angular velocity) for original acc_mag ...
 Removing gravity and calculating linear acceleration features...
  Calculating angular velocity from quaternion derivatives...
  Calculating angular distance between successive quaternions...
  IMU (incl. engineered & derivatives) 17 | THM + Aggregated TOF 25 | total 42 features
  Building sequences with aggregated TOF and preparing data for scaler...
  Fitting StandardScaler...
  Scaling and padding sequences...
  Splitting data and preparing for training...


In [ ]:
model = TwoBranchGestureModel(imu_dim=imu_dim_final, tof_dim=tof_thm_aggregated_dim_final, n_classes=len(le.classes_))
model.to(device)

from torch.optim.lr_scheduler import CosineAnnealingLR
optimizer = torch.optim.AdamW(model.parameters(), lr=LR_INIT, weight_decay=WD)
scheduler = CosineAnnealingLR(optimizer, T_max=EPOCHS)  # T_max = number of epochs
weights_tensor = torch.tensor(cw_vals, dtype=torch.float32).to(device)
loss_fn = nn.CrossEntropyLoss(weight=weights_tensor, label_smoothing=0.1)

best_val_acc = 0
patience, patience_counter = 10, 0
EPOCHS = EPOCHS
counter = 0
ud = []
steps = []
lrs = []


for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    correct = 0         # <-- reset here
    total = 0           # <-- reset here
    for xb, yb in tqdm.tqdm(train_loader):
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        preds = model(xb)
        loss = loss_fn(preds, yb)
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)  # optional
        # Track parameter-wise update dynamixs
        with torch.no_grad():
            updates = []
            for p in model.parameters():
                if p.grad is not None and p.data.std() > 0:
                    ratio = (LR_INIT * p.grad.std() / (p.data.std() + 1e-8))
                    ud.append(ratio.log10().item())  # log10 for readability
                    
        optimizer.step()
        scheduler.step()

        # accumulate train data
        total_loss += loss.item()
        lrs.append(scheduler.get_last_lr()[0])
        steps.append(epoch * BATCH_SIZE + counter + 1)

        pred_labels = preds.argmax(dim=1)
        yb = yb.argmax(dim=1)
        correct += (pred_labels == yb).sum().item()
        total += yb.size(0)
    
    train_acc = correct / total
    print(f"Epoch {epoch} | Train Loss: {total_loss / len(train_loader):.4f} | Train Acc: {train_acc:.4f}")

    # Validation
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for xb, yb in val_loader:
            xb, yb = xb.to(device), yb.to(device)
            preds = model(xb)
            pred_labels = preds.argmax(1)
            true_labels = yb.argmax(1)
            correct += (pred_labels == true_labels).sum().item()
            total += yb.size(0)
    val_acc = correct / total
    print(f"Epoch {epoch} | Val Acc: {val_acc:.4f}")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        patience_counter = 0
        torch.save(model.state_dict(), EXPORT_DIR / "best_model.pt")
        print(f"Best Acc so far {best_val_acc:.4f}")
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print("Early stopping triggered.")
            break


100%|██████████| 26/26 [00:01<00:00, 13.32it/s]


Epoch 0 | Train Loss: 2.7277 | Train Acc: 0.2242
Epoch 0 | Val Acc: 0.3121
Best Acc so far 0.3121


100%|██████████| 26/26 [00:01<00:00, 15.05it/s]


Epoch 1 | Train Loss: 2.2773 | Train Acc: 0.3451
Epoch 1 | Val Acc: 0.4414
Best Acc so far 0.4414


100%|██████████| 26/26 [00:01<00:00, 15.22it/s]


...

Epoch 66 | Train Loss: 1.2408 | Train Acc: 0.8952
Epoch 66 | Val Acc: 0.7082


100%|██████████| 26/26 [00:01<00:00, 14.98it/s]


Epoch 67 | Train Loss: 1.2410 | Train Acc: 0.8926
Epoch 67 | Val Acc: 0.7088


100%|██████████| 26/26 [00:01<00:00, 14.98it/s]


Epoch 68 | Train Loss: 1.2534 | Train Acc: 0.8891
Epoch 68 | Val Acc: 0.6597
Early stopping triggered.


In [12]:
ud

[]

In [11]:
#### Visualize UD
import matplotlib.pyplot as plt

# Flatten and average across steps for each parameter group
ud_tensor = torch.tensor(ud)  # shape: (steps, n_param_groups)
avg_updates = ud_tensor.mean(dim=0)

plt.bar(range(len(avg_updates)), avg_updates)
plt.title("Avg log10(update/weight std) per parameter tensor")
plt.xlabel("Param Group Index")
plt.ylabel("log10(update/weight)")
plt.show()

TypeError: len() of a 0-d tensor

In [10]:
model.load_state_dict(torch.load(EXPORT_DIR / "best_model.pt"))
model.eval()
preds_val = []

with torch.no_grad():
    for xb, _ in val_loader:
        xb = xb.to(device)
        logits = model(xb)
        preds_val.append(logits.argmax(1).cpu())

preds_val = torch.cat(preds_val).numpy()
true_val_int = y_val.argmax(1).numpy()

# Evaluation
from cmi_2025_metric_copy_for_import import CompetitionMetric

h_f1 = CompetitionMetric().calculate_hierarchical_f1(
    pd.DataFrame({'gesture': le.classes_[true_val_int]}),
    pd.DataFrame({'gesture': le.classes_[preds_val]})
)
print("Hold-out H-F1 =", round(h_f1, 4))

Hold-out H-F1 = 0.3602
